In [ ]:
pip install langchain
pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
pip install openai
pip install langchain-openai
pip install pypdf
pip install faiss-cpu
pip install PyPDF2
pip install pdfminer.six

In [1]:
import bs4
import os
import config
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.vectorstores import FAISS
#from langchain_community.document_loaders import PyPDFDirectoryLoader

In [2]:
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY
os.environ["LANGSMITH_API_KEY"] = config.LANGSMITH_API_KEY

In [3]:
llm = ChatOpenAI()
#chat_model = ChatOpenAI()

In [ ]:
# cargar PDFs desde un directorio

#loader = PyPDFDirectoryLoader("example_data/")
# docs = loader.load()

In [15]:
# Se carga el archivo de condiciones generales y el de condiciones particulares para un caso
#loader_ccgg = PyPDFLoader("CCGG_REALE.pdf")
loader_ccpp = PyPDFLoader("CCPP_REALE_ALE.pdf")
#pages_ccgg = loader_ccgg.load_and_split()
pages_ccpp = loader_ccpp.load_and_split()

In [ ]:
#pages_ccgg[0]

In [16]:
pages_ccpp[0]

Document(page_content='Reale SegurosGenerales,S.A.condomiciliosocialenC/PrÝncipedeVergara125,28002,Madrid,conCIFA-78520293inscritaenelRegistroMercantildeMadrid,tomo7795general,6748delaSecci¾n3¬delLibrodeSociedades,Folio153,Hojan║76036-1,inscripci¾n1¬\nRSG0081703v02ES160P20230824REALE HOGAR\nCertificado deseguro\nN║POLIZA/SPTO.: 8172300068708 /0\nREALE HOGAR PRINCIPAL\n71/\nwww.reale.es\nCENTRO REALE 6052\nBURGOS\nAVDA. CID CAMPEADOR, 71║-B\n09003 BURGOS\nFax: 947273678\nREALE.BURGOS@REALE.ES\nMEDIADOR /033436 /\nCORREDOR\nASPURU CORREDURIA DESEGUROS, S.L\nCLVIRGEN DEL MANZANO, 1\n09004 BURGOS\nTfno: 947063737\naspuru@aspurucorreduria.com\nASEGURADOR\nA78520293 CIF:REALE SEGUROS GENERALES, S.A. PRINCIPE DEVERGARA, 125, 28002 MADRID\nTOMADOR\nPEÐA MONTERO, ANA\n28045 MADRIDNIF: 47700596E\nTelÚfono: 617205988CLJAIME ELCONQUISTADOR, 15, 6CDE\nEmail: a.saizj06@gmail.com\nASEGURADO\nPEÐA MONTERO, ANA\nNIF: 47700596E\nEFECTO DEL SEGURO, DURACIËN YFORMA DEPAGO\nEfecto:\nVencimiento:24/11/2023\

In [17]:
# búsqueda de similitud de documentos basada en vectores de características

# se crea índice a partir del documento de ccpp. OpenAIEmbeddings se utiliza para representar los docs como vectores de características
faiss_index_ccpp = FAISS.from_documents(pages_ccpp, OpenAIEmbeddings())

# se realiza búsqueda de similitud
docs = faiss_index_ccpp.similarity_search("¿Cuál es el importe del recibo para la póliza 8172300068708‬?")

# se itera sobre los documentos devueltos previamente y se imprime información sobre ellos.
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

5: Reale SegurosGenerales,S.A.condomiciliosocialenC/PrÝncipedeVergara125,28002,Madrid,conCIFA-78520293inscritaenelRegistroMercantildeMadrid,tomo7795general,6748delaSecci¾n3¬delLibrodeSociedades,Folio153,Hojan║76036-1,inscripci¾n1¬
RSG0081703v02ES160P20230824REALE HOGAR
Certificado deseguro
www.reale.es
2: Reale SegurosGenerales,S.A.condomiciliosocialenC/PrÝncipedeVergara125,28002,Madrid,conCIFA-78520293inscritaenelRegistroMercantildeMadrid,tomo7795general,6748delaSecci¾n3¬delLibrodeSociedades,Folio153,Hojan║76036-1,inscripci¾n1¬
RSG0081703v02ES160P20230824REALE HOGAR
Certificado deseguro
www.reale.es
0: Reale SegurosGenerales,S.A.condomiciliosocialenC/PrÝncipedeVergara125,28002,Madrid,conCIFA-78520293inscritaenelRegistroMercantildeMadrid,tomo7795general,6748delaSecci¾n3¬delLibrodeSociedades,Folio153,Hojan║76036-1,inscripci¾n1¬
RSG0081703v02ES160P20230824REALE HOGAR
Certificado deseguro
N║POLIZA/SPT
6: Certificado deseguroREALE HOGAR
www.reale.es77/N║POLIZA/SPTO.: 8172300068708 /0
POL═T

In [18]:
# se dividen los documentos en fragmentos de 1000 caracteres con 200 caracteres de superposición entre fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [19]:
len(splits)

11

In [20]:
len(splits[0].page_content)

508

In [21]:
splits[10].metadata

{'source': 'CCPP_REALE_ALE.pdf', 'page': 6}

In [22]:
# se almacenan los chunks en una base de datos de vectores
vectorstore = Chroma.from_documents(documents = splits, embedding = OpenAIEmbeddings())

In [23]:
# la lógica de la app es que se responda la pregunta de un usuario según lo almacenado en el vectorstore
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
retrieved_docs = retriever.invoke("¿Cuál es el importe del recibo para la póliza 8172300068708‬?")

In [24]:
len(retrieved_docs)

6

In [25]:
print(retrieved_docs[0].page_content)

Email: a.saizj06@gmail.com
ASEGURADO
PEÐA MONTERO, ANA
NIF: 47700596E
EFECTO DEL SEGURO, DURACIËN YFORMA DEPAGO
Efecto:
Vencimiento:24/11/2023
24/11/2024Duraci¾n del Seguro:
Forma de Pago:ANUAL RENOVABLE
ANUAL0h.del
0h.del
PERIODO DECERTIFICACIËN DEL SEGURO
Efecto:
Vencimiento:24/11/2023
24/11/20240h.del
0h.del
IMPORTE DEL RECIBO YM╔TODO DEPAGO
Importe delRecibo Periodo Prima Consorcio D.G.S. Impuestos Total
Anual86,05 Inicial
97,60 Ç
CARTA RECIBO0,13 9,21 Del24-11-2023 al24-11-2024 2,21 97,60 Ç
MÚtodo depago:


In [26]:
prompt = hub.pull("rlm/rag-prompt")

In [27]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [28]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [29]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
for chunk in rag_chain.stream("¿Cuál es el importe del recibo para la póliza 8172300068708‬?"):
    print(chunk, end = "", flush = True)

El importe del recibo para la póliza 8172300068708 es de 97,60 €. El periodo de cobertura va del 24-11-2023 al 24-11-2024. El método de pago no está especificado en el contexto proporcionado.

In [ ]:
# cleanup
#vectorstore.delete_collection()